In [1]:
%pip install ultralytics

from ultralytics import YOLO

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/Users/ikmalbasirun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Load the models
modelv5_tuning = YOLO("yolov5nu.pt")   # these first three models are for hyperparameter tuning
modelv8_tuning = YOLO("yolov8n.pt")
modelv11_tuning = YOLO("yolo11n.pt")

# Tuning

In [3]:
import random

search_space = {
    "lr0": (1e-5,1e-4,1e-3,1e-2,1e-1),
    "batch": [4,6,8,10,12,14,16,18,20]
}

#Save the hyperparameter values used in the tuning
lr0_dict = {}
batch_dict = {}

for i in range(15):
  lr0 = search_space["lr0"][random.randint(0,4)]
  batch = search_space["batch"][random.randint(0,8)]

  lr0_dict[i] = lr0
  batch_dict[i] = batch

       #change the model version here
  modelv8_tuning.train( 
    data='wisard_dataset_thermal.yaml', #state whether it's RGB or THERMAL 
    epochs=5,
    imgsz=640,
    project='runs/v8_thermal', # change runs/VERSION_MODALITY
    batch=batch,
    lr0=lr0,
    optimizer = 'SGD',
    device='mps', # mpu is for MAC, change to 'auto' or remove this if you're using Windows
    workers=4
  )


New https://pypi.org/project/ultralytics/8.3.127 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.91 🚀 Python-3.9.6 torch-2.6.0 MPS (Apple M4 Pro)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=wisard_dataset_thermal.yaml, epochs=5, time=None, patience=100, batch=10, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=4, project=runs/v8_thermal, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, 

train: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/thermal/labels.cache... 1128 images, 100 backgrounds, 0 corrupt: 100%|██████████| 1128/1128 [00:00<?, ?it/s]

train: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/thermal/images/DJI_0402.mp4_00001.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/thermal/images/DJI_0402.mp4_00002.jpg: 3 duplicate labels removed
train: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/thermal/images/DJI_0402.mp4_00003.jpg: 3 duplicate labels removed
train: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/thermal/images/DJI_0402.mp4_00004.jpg: 3 duplicate labels removed
train: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/thermal/images/DJI_0402.mp4_00005.jpg: 3 duplicate labels removed



val: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/thermal/labels.cache... 375 images, 35 backgrounds, 0 corrupt: 100%|██████████| 375/375 [00:00<?, ?it/s]

Plotting labels to runs/v8_thermal/train/labels.jpg... 


...

WARNING ⚠️ NMS time limit 2.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  75%|███████▌  | 24/32 [01:54<01:00,  7.62s/it]

WARNING ⚠️ NMS time limit 2.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  75%|███████▌  | 24/32 [01:57<00:39,  4.91s/it]


KeyboardInterrupt: 

# Tuning Results

In [ ]:
import pandas as pd

result_paths = ["train4","train42","train422","train4222","train42222"] #state all output folders from training done during tuning

results_df = pd.DataFrame()

for i, path in enumerate(result_paths): #accesses the results.csv file from each output folders
  # Path to the results.csv file from tuning
  results_path = f'runs/v5_RGB/{path}/results.csv' #change the folder version and modality according to what you've tuned

  # Load results
  df = pd.read_csv(results_path)

  #Obtain the saved hyperparameter values during tuning process
  df['lr0'] = lr0_dict[i]
  df['batch'] = batch_dict[i]

  #Concat to results_df
  df['run_name'] = path  # Track which run it came from
  results_df = pd.concat([results_df, df], ignore_index=True)


# Sort by recall (metrics/recall(B))
results_df_sorted = results_df.sort_values(by='metrics/recall(B)', ascending=False) #changed metrics to recall

In [ ]:
results_df_sorted.head(10)